# Weekly ranking for newsletter

**Goal**: get the weekly emission rankings of 
- subnets
- validators

In [1]:
import pandas as pd
import bittensor as bt
from substrateinterface import SubstrateInterface
subtensor = bt.subtensor('archive')

/Users/gustave/miniconda3/envs/bittensor/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/Users/gustave/miniconda3/envs/bittensor/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(


In [2]:
substrate = SubstrateInterface(url="wss://archive.chain.opentensor.ai:443/")

In [3]:
block = substrate.get_block()['header']['number']
print(block)

3120660


### Subnets

In [4]:
# live_emissions
current_emissions = {}
for i in range(0, 35):
    emission = subtensor.get_emission_value_by_subnet(i, block)
    current_emissions[i] = float(emission)*100

current_emissions = pd.DataFrame.from_dict(current_emissions, orient='index', columns=['Emissions %'])
current_emissions = current_emissions.rename_axis('Subnet')

In [5]:
# emissions 7 days ago
# je dois aller chercher le dernier bloc et enlever 7 * 7200 = 50 400 blocks

emissions_7daysago = {}
block_7daysago = block - 7200*7
for i in range(0, 35):
    emission = subtensor.get_emission_value_by_subnet(i, block_7daysago)
    emissions_7daysago[i] = float(emission)*100

emissions_7daysago = pd.DataFrame.from_dict(emissions_7daysago, orient='index', columns=['Past Emissions %'])
emissions_7daysago = emissions_7daysago.rename_axis('Subnet')
emissions_df = pd.concat([current_emissions, emissions_7daysago], axis=1)

In [6]:
# Change in rank
current_emissions_ranked = current_emissions
emissions_7daysago_ranked = emissions_7daysago
current_emissions_ranked = current_emissions_ranked.sort_values(by='Emissions %', ascending=False)
emissions_7daysago_ranked = emissions_7daysago_ranked.sort_values(by='Past Emissions %', ascending=False)
current_emissions_ranked.reset_index(inplace=True)
emissions_7daysago_ranked.reset_index(inplace=True)

In [7]:
difference = {}
for i in range(0, 35):
    current_rank = current_emissions_ranked[current_emissions_ranked['Subnet'] == i].index.tolist()[0]
    previous_rank = emissions_7daysago_ranked[emissions_7daysago_ranked['Subnet'] == i].index.tolist()[0]
    difference[i] = previous_rank - current_rank

difference_df = pd.DataFrame.from_dict(difference, orient='index', columns=['Difference'])
difference_df = difference_df.rename_axis('Subnet')

subnet_emissions = pd.concat([emissions_df, difference_df], axis=1)


In [8]:
sum = subnet_emissions['Emissions %'].sum()
print(sum)

99.9796739


### Validators

Do we want a ranking of all the validators or only the root network ones ? <br>
What should we rank them with, vtrust or net return ? -> People don't care about vtrusts, only about the % returns of validators so they know where to stake

##### Method by validator
1. Get top 64 validators hotkeys from the rootnetwork metagraph
2. Look in every subnet and add how much they earn from each one/day (subnetEmissions * 0.41* dividends*7200)
3. Make the ranking

In [9]:
root_metagraph = bt.metagraph(netuid=0)

2024-06-06 18:43:22.892 |     WARNING      |  - We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network. - 
2024-06-06 18:43:22.893 |     WARNING      |  - In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it. - 


In [10]:
# get top 64 validators hotkeys
top_validators = root_metagraph.neurons
df = pd.DataFrame(top_validators)
top_validators_hotkeys = df.hotkey

In [11]:
# current APR
daily_payout = {key: 0 for key in top_validators_hotkeys}
for i in range(1, 34):
    metagraph = bt.metagraph(i)
    emissions = float(subtensor.get_emission_value_by_subnet(i, block))
    neurons = pd.DataFrame(metagraph.neurons)
    # print(neurons)
    for val in top_validators_hotkeys:
        if val in neurons.hotkey.tolist():
            dividends = neurons.loc[neurons['hotkey'] == val, 'dividends'].iloc[0]
            daily_payout[val] += dividends * emissions * 0.41 * 7200
        else:
            daily_payout[val] += 0
    # print(daily_payout)

2024-06-06 18:43:33.834 |     WARNING      |  - We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network. - 
2024-06-06 18:43:33.834 |     WARNING      |  - In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it. - 
2024-06-06 18:43:47.371 |     WARNING      |  - We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network. - 
2024-06-06 18:43:47.371 |     WARNING      |  - In a future release, local subtensor will become the default endpoint. To get ahead of this change, please run a local subtensor node and point to it. - 
2024-06-06 18:43:59.379 |     WARNING      |  - We strongly encourage running a local subtensor node whenever possible. This increases decentralization and resilience of the network. - 
2024-06-06 18:43:59.379 |     WARNING 

In [12]:
validators_df = pd.DataFrame(daily_payout, index=[0]).T
validators_df = validators_df.rename(columns={0: 'daily reward'})
validators_df = validators_df.rename_axis('hotkey')

In [13]:
stake_df = df[['hotkey', 'total_stake']].set_index('hotkey')
stake_df['total_stake'] = [float(element) for element in stake_df.total_stake.tolist()]

In [14]:
validators_df = validators_df.join(stake_df)

In [15]:
validators_df['APR'] = ((validators_df['daily reward'] * 365) / validators_df['total_stake']) * 100

In [16]:
validators_df['stake share'] = (validators_df['total_stake'] / validators_df['total_stake'].sum())*100

In [17]:
# mise en page pour google sheet
validators_df = validators_df[validators_df['APR'] > 15].sort_values('APR', ascending=False)
validators_df = validators_df.round(2)
validators_df = validators_df.drop(columns=['daily reward', 'total_stake'])

### Download to CSV

In [18]:
# don't forget to change the name !!
from IPython.display import FileLink
validators_df.to_csv('validators_ranking_06_06.csv', index=True)
FileLink('validators_ranking_06_06.csv')

/Users/gustave/Desktop/crypto/programming/bittensor/network-analysis/rankings/validators_ranking_06_06.csv